In [5]:
from flask import Flask, render_template, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
#import the model and pipeline as function to activate from html

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


app = Flask(__name__)

#restaurant input
@app.route("/")
def index_1st_stp():
    
    return render_template('index.html')


@app.route("/get_restaurant", methods=["GET", "POST"])
def restaurant_chat():
    msg = request.form["msg"]
    input = msg
    return send_restaurant(input)


def send_restaurant(text):
    #send to pipeline for selection of reviews on restaurant
    return 'Please enter dish query'
    
#request input

@app.route("/get_request", methods=["GET", "POST"])
def request_chat():
    msg = request.form["msg"]
    input = msg
    return get_Request_response(input)


def get_Request_response(text):

    # Let's chat for 5 lines
    for step in range(5):
        # encode the new user input, add the eos_token and return a tensor in Pytorch
        new_user_input_ids = tokenizer.encode(str(text) + tokenizer.eos_token, return_tensors='pt')

        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

        # generated a response while limiting the total chat history to 1000 tokens, 
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

        # pretty print last ouput tokens from bot
        return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

@app.route("/reset", methods=["POST"])
def reset():
    return jsonify(success=True)

if __name__ == '__main__':
    app.run()